# How to get gene sequences from a set of genbank records

I want to perform a multiple sequence alignment on a singular gene (glycoprotein E) using sequences from a BioProject containing multiple genbank records: 

[BioProject PRJNA344504](https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJNA344504) 
from the publication [Zika virus evolution and spread in the Americas](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5563848/#SD1)

Genbank records, however, are complete records containing all the sequence information. How do I to remove only the information that I need (in this case all the glycoprotein E sequences) and copy them into a new file?

After downloading all the records into one file, it will be opened in the following script, which will then:

1. parse through each genbank record,
2. search for the identifying feature and qualifier, and if there is a match, 
3. find that feature's sequence, and
4. save it to a new file in FASTA format

There are four arguments that need to be specified when calling genbank_qualifier_hunter:

gb_file: the location and name of your genbank file
feature_type: the label of the required feature in the genbank records
qualifier: the identifying label within that feature object
qualifier_value: the specific name of your gene or area of interest

In addition to the new file that is created, you will receive a message telling you how many records were copied into your new file. 

At this point I usually manually inspect the file before applying my alignment.


In [1]:
from Bio import SeqIO

new_file = open('env-only_PRJNA344504.gb', 'w+')

def genbank_qualifier_hunter(gb_file, feature_type, qualifier, qualifier_value):
    
    answer = ''
    count = 0  
    for gb_record in SeqIO.parse(open(gb_file,'r'), 'genbank'):
        
        for (index, feature) in enumerate(gb_record.features):
            if feature.type == feature_type:
                if qualifier in feature.qualifiers:
                    value = feature.qualifiers.get(qualifier)
                    if value == qualifier_value:
                        sequence = (feature.location.extract(gb_record.seq))                       
                        answer+=">" + gb_record.id + ' | ' + gb_record.description + ' ' + str(feature.location) + '\n'
                        answer+=sequence + '\n' + '\n'
                        count += 1
    new_file.write(str(answer))
    new_file.close()
    return count

total_records = genbank_qualifier_hunter('Zika Virus/PRJNA344504.gb', 'mat_peptide', 'product', ['envelope'])
print('Your new file ' + str(new_file) + ' contains ' + str(total_records) + ' fasta records.')

Your new file <closed file 'env-only_PRJNA344504.gb', mode 'w+' at 0x1046360c0> contains 109 fasta records.
